<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Domain_Pre_training_Compile_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This takes a 50k review (8000 row condensed) dataset and tries compiling to increase training time for the A100.

# Results


# Compile info
* Train time for 2 epochs: 303.4892 (includes compile time?)
* For 1 epoch after compile run: 87.8487
* Equals 568 / reviews per second
* At 568/reviews per second, expect training to take **2.45 hours for 5M (20%)** of dataset.

# No Compile Info
At 192/reviews per second, expect training to take **7.2 hours for 5M (20%)** of dataset.

Training time: 260.73 seconds
192 reviews / second.

# Details
* Don't push model to hub, only experimenting
* Uses batch size of 34 and gradient accumulation of 10
* Uses two epochs to check time on second epoch





# Setup

In [1]:
!pip install datasets
!pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.1 MB/s eta 0:00:00


In [3]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

In [4]:
from time import time

In [5]:
from datasets import load_dataset
dataset = load_dataset("BigTMiami/amazon_25M_50_000_condensed")
print(dataset)
print_gpu_utilization()

Generating train split:   0%|          | 0/8277 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/855 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8277
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 855
    })
})
GPU 448 MB


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512
GPU 448 MB


In [7]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_config(config)
print_gpu_utilization()

GPU 448 MB


# Train

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="amazon_pretraining_tuning",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=34, # 346,m - TRYING SMALLER BATCH SIZE
    per_device_eval_batch_size=34, #346,m - TRYING SMALLER BATCH SIZE
    num_train_epochs=2, # CREATED TWO EPOCHS TO CHECK TIME AFTER COMPILE
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    torch_compile=True,  # THIS IS THE POTENTAIL CHANGE
    # load_best_model_at_end=True,
    # push_to_hub=True,
)
print_gpu_utilization()

GPU 451 MB


In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


GPU 1399 MB


In [10]:
start_time = time()
results = trainer.train()
training_time = time() - start_time
print(f"Training time: {training_time:.2f} seconds")
print(f"torch_compile:{trainer.args.torch_compile}")
print(f"gradient_accumulation_steps:{trainer.args.gradient_accumulation_steps}")
print(f"per_device_train_batch_size:{trainer.args.per_device_train_batch_size}")
print_gpu_utilization()


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss
1,No log,7.104016
2,No log,7.063349


Training time: 304.72 seconds
torch_compile:True
gradient_accumulation_steps:1
per_device_train_batch_size:34
GPU 39223 MB


In [14]:
print(training_args.logging_strategy)

IntervalStrategy.STEPS


In [11]:
print(results)

TrainOutput(global_step=488, training_loss=7.241002817622951, metrics={'train_runtime': 303.4892, 'train_samples_per_second': 54.546, 'train_steps_per_second': 1.608, 'total_flos': 4358096479401984.0, 'train_loss': 7.241002817622951, 'epoch': 2.0})


In [15]:
trainer.state.log_history

[{'eval_loss': 7.104015827178955,
  'eval_runtime': 4.3773,
  'eval_samples_per_second': 195.326,
  'eval_steps_per_second': 5.94,
  'epoch': 1.0,
  'step': 244},
 {'eval_loss': 7.06334924697876,
  'eval_runtime': 3.7858,
  'eval_samples_per_second': 225.844,
  'eval_steps_per_second': 6.868,
  'epoch': 2.0,
  'step': 488},
 {'train_runtime': 303.4892,
  'train_samples_per_second': 54.546,
  'train_steps_per_second': 1.608,
  'total_flos': 4358096479401984.0,
  'train_loss': 7.241002817622951,
  'epoch': 2.0,
  'step': 488}]

In [16]:
training_args = TrainingArguments(
    output_dir="amazon_pretraining_tuning",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=34, # 346,m - TRYING SMALLER BATCH SIZE
    per_device_eval_batch_size=34, #346,m - TRYING SMALLER BATCH SIZE
    num_train_epochs=1, # Just one epoch for this test, already compiled
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    torch_compile=True,  # THIS IS THE POTENTAIL CHANGE
    logging_steps=50, # SHOULD BE MUCH HIGHER FOR LONG RUNS, JUST TESTING
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
print_gpu_utilization()

GPU 39223 MB


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
start_time = time()
results = trainer.train()
training_time = time() - start_time
print(f"Training time: {training_time:.2f} seconds")
print(f"torch_compile:{trainer.args.torch_compile}")
print(f"gradient_accumulation_steps:{trainer.args.gradient_accumulation_steps}")
print(f"per_device_train_batch_size:{trainer.args.per_device_train_batch_size}")
print(f"training_args.logging_strategy: {training_args.logging_strategy}")
print(f"training_args.logging_steps: {training_args.logging_steps}")
print_gpu_utilization()

print(results)
trainer.state.log_history

Epoch,Training Loss,Validation Loss
1,7.091200,7.094724


Checkpoint destination directory amazon_pretraining_tuning/checkpoint-244 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Training time: 88.59 seconds
torch_compile:True
gradient_accumulation_steps:1
per_device_train_batch_size:34
training_args.logging_strategy: steps
training_args.logging_steps: 50
GPU 39737 MB
TrainOutput(global_step=244, training_loss=7.100118918497055, metrics={'train_runtime': 87.8487, 'train_samples_per_second': 94.219, 'train_steps_per_second': 2.778, 'total_flos': 2179048239700992.0, 'train_loss': 7.100118918497055, 'epoch': 1.0})


[{'loss': 7.1118,
  'grad_norm': 0.7194158434867859,
  'learning_rate': 0.0004235807860262009,
  'epoch': 0.2,
  'step': 50},
 {'loss': 7.1026,
  'grad_norm': 0.7031736373901367,
  'learning_rate': 0.000314410480349345,
  'epoch': 0.41,
  'step': 100},
 {'loss': 7.0938,
  'grad_norm': 0.5378099083900452,
  'learning_rate': 0.0002052401746724891,
  'epoch': 0.61,
  'step': 150},
 {'loss': 7.0912,
  'grad_norm': 0.48665812611579895,
  'learning_rate': 9.606986899563319e-05,
  'epoch': 0.82,
  'step': 200},
 {'eval_loss': 7.094723701477051,
  'eval_runtime': 3.8219,
  'eval_samples_per_second': 223.712,
  'eval_steps_per_second': 6.803,
  'epoch': 1.0,
  'step': 244},
 {'train_runtime': 87.8487,
  'train_samples_per_second': 94.219,
  'train_steps_per_second': 2.778,
  'total_flos': 2179048239700992.0,
  'train_loss': 7.100118918497055,
  'epoch': 1.0,
  'step': 244}]

In [18]:
!ls

amazon_pretraining_tuning  sample_data


In [20]:
!cd amazon_pretraining_tuning/

In [30]:
!ls -l amazon_pretraining_tuning/checkpoint-244/

total 1461532
-rw-r--r-- 1 root root       672 Apr  8 18:42 config.json
-rw-r--r-- 1 root root 498813948 Apr  8 18:42 model.safetensors
-rw-r--r-- 1 root root 997748346 Apr  8 18:42 optimizer.pt
-rw-r--r-- 1 root root     14244 Apr  8 18:42 rng_state.pth
-rw-r--r-- 1 root root      1064 Apr  8 18:42 scheduler.pt
-rw-r--r-- 1 root root      1301 Apr  8 18:42 trainer_state.json
-rw-r--r-- 1 root root      4920 Apr  8 18:42 training_args.bin


In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()